# Multi-Label Classification
### Random Forest Classifier

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

import sklearn
from sklearn import metrics, model_selection, tree
from sklearn.preprocessing import MinMaxScaler

#### Carga de data sets
https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html
https://www.analyticsvidhya.com/blog/2017/08/introduction-to-multi-label-classification/

In [2]:
# Train data
X_train = pd.read_csv('x_train_smote_oversample.csv')
y_train = pd.read_csv('y_train_smote_oversample.csv')

# Test data
X_test = pd.read_csv('x_test_smote_oversample.csv')
y_test = pd.read_csv('y_test_smote_oversample.csv')

# Validation data
X_validation = pd.read_csv('x_validation_smote_oversample.csv')
y_validation = pd.read_csv('y_validation_smote_oversample.csv')

In [3]:
from sklearn.ensemble import RandomForestClassifier

In [4]:
clf = RandomForestClassifier(max_depth=1, random_state=6)

In [ ]:
clf.fit(X_train, y_train)

In [ ]:
predictions = clf.predict(X_test)

In [ ]:
clf.score(X_test, y_test)

In [ ]:
print(metrics.accuracy_score(y_test, predictions))
print('Matriz de confusion Random Forest Classifier\n', metrics.confusion_matrix(y_test, predictions))
print(metrics.classification_report(y_test, predictions, target_names=['0', '1', '2', '3']))

In [ ]:
metrics.plot_roc_curve(clf, X_test, y_test)
plt.show()

In [ ]:
import joblib
joblib.dump(scaler,'escalador.pkl')

In [ ]:
joblib.dump(clf, 'modelo2.h5')